## Important Dependencies with their required versions


In [ ]:
# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb
# ! pip install text_generation
# ! pip install sentence-transformers
# ! pip install python-dotenv
# ! pip install transformers
# ! pip install torch
# ! pip install langchain_community
# ! pip install pandas
# ! pip install astunparse
# ! pip install faiss-cpu
# ! pip install flask_cors
# ! pip install openai
# ! pip install pandas
# ! pip install sentence-transformers
# ! pip install datasets
# ! pip install langchain_openai

### LLAMA INDEX

In [3]:
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api
%pip install "transformers[torch]" "huggingface_hub[inference]"
%pip install llama-index
%pip install llama-index-llms-text-generation-inference

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 5.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 29.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from llama_index.core import PromptTemplate

GenerateTestTemplate = """You are a python expert and your task is: Given the following description and python code:
Description:
{description}
Code:
'''python
{code}'''
Generate a class that contains at least 7 unit tests (where each test has only one assertion) written in python that acheive high coverage to find bugs, runtime errors or logical errors in the code according to the description and include any required imports.
Make sure to include the unit test call unittest.main() to run the tests. Do not include any import for the code under test.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```" respectively:"""

qa_template = PromptTemplate(GenerateTestTemplate)

prompt = qa_template.format(description="Write a function that takes a list of integers and returns the sum of the list.", code="def sum_list(lst):\n    return sum(lst)")
print(prompt)

You are a python expert and your task is: Given the following description and python code:
Description:
Write a function that takes a list of integers and returns the sum of the list.
Code:
'''python
def sum_list(lst):
    return sum(lst)'''
Generate a class that contains at least 7 unit tests (where each test has only one assertion) written in python that acheive high coverage to find bugs, runtime errors or logical errors in the code according to the description and include any required imports.
Make sure to include the unit test call unittest.main() to run the tests. Do not include any import for the code under test.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```" respectively:


In [7]:
import os
from typing import List, Optional

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

URL = "mistralai/Mixtral-8x7B-Instruct-v0.1"
remotely_run = HuggingFaceInferenceAPI(
    model_name=URL, token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

completion_response = remotely_run.complete("To infinity, and")
print(completion_response)

 beyond!

That’s the message from the European Space Agency, which has announced plans to build a “village” on the moon.

The ESA’s director general, Johann-Dietrich Woerner, said the agency is working on a plan to build a moon base that would serve as a stepping stone for future missions to Mars and beyond.

“We think we can establish a village on the moon within the next 20 years,” Woerner told the BBC.

The ESA is working with international partners, including NASA, to develop the technology needed to build the lunar village.

The base would be built using 3D printing technology, and would be powered by solar energy.

It would also serve as a research center, where scientists could study the moon’s geology and atmosphere.

Woerner said the lunar village would be a “first step” towards establishing a human presence on Mars.

“We want to go to Mars, but before going to Mars we have to learn how to live on another celestial body,” he said.

The ESA is not the only organization with pla

## Imports and Setup and Initializing Agent

In [1]:
# auto reload instead of having to restart each time
%load_ext autoreload
%autoreload 2

In [1]:
# from Imports import *
# try:
from Configuration import *
from Benchmark.TestGenerator import *
from Benchmark.TestFix import *
from Benchmark.DecisionMaker import *
from Benchmark.BugFixBench import *
from Benchmark.BugFix import *

# llm_arb, chat_model_arb = InitializeModelArbiter(os.environ["HUGGINGFACEHUB_API_TOKEN"],repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1')
# with open(r"Datasets/atcoder_problem_test_cases_description.jsonl", "r") as f:
#     data = f.read()
#     jsonObj = json.loads(data)
# jsonObj=pd.read_json(path_or_buf="Datasets/atcoder_problem_test_cases_description.jsonl")
# jsonObj=pd.read_json(path_or_buf="Datasets/cleanedhumaneval.json")
jsonObj = HEval_JsonObj
testGenerator = TestGenerator(GenUnitTestChain, db, jsonObj, globals())
# testGenerator.isHumanEval = True
testRegenerator = TestFix(UnitTestFeedbackChain,True,globals())
# judgeGenerator = DecisionMaker(judgeChain,globals())
# bugFixGenerator = BugFix(bugFixChain,True,globals())
# bugFixGeneratorben = BugFixBench(bugFixChainben,True,globals())
# except Exception as e:
#     print(e)
#     exit(-1)

/home/omare/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/omare/venv/lib/python3.12/site-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


## Generate Unit Tests from Code

In [2]:
testGenerator.generate(55, 56)

Running Test Case  55
Test example 55 failed

Number of Ran Tests :  7
Number of failed Tests :  7
Number of error Tests :  0
Number of Succeeded Test :  0

 End of Test Cases 

Total succeeded examples :  0
Total failed examples :  1
Total  testcases :  7
Total succeeded testcases :  0
Total failed testcases :  7
Total error testcases :  0
Incomplete Responses are :  0
Incomplete Responses are  0
API Errors are :  0
Tests to repeat :  0


### Test Fix

In [5]:
testRegenerator = TestFix(UnitTestFeedbackChain,True,globals())

In [2]:
# judgeGenerator = DecisionMaker(judgeChain,globals())
testRegenerator.generate(30)

True
Running Example  30 

Example 30  has already passed
Running Example  31 

Example 31  has already passed
jyjy
Running Example  32 

Test example 32 failed

Number of Ran Tests :  7
Number of failed Tests :  4
Number of error Tests :  1
Number of Succeeded Test :  2
ERROR! Session/line number was not unique in database. History logging moved to new session 73
Running Example  33 

Test example 33 failed

Number of Ran Tests :  8
Number of failed Tests :  2
Number of error Tests :  0
Number of Succeeded Test :  6
Running Example  34 

Test example 34 succeeded

Number of Ran Tests :  4
Number of Succeeded Test :  4
Number of Succeeded Test :  0
Running Example  35 

Test example 35 failed due to syntax or indentation or timeout

Number of Ran Tests :  0
Number of failed Tests :  0
Number of error Tests :  1
Number of Succeeded Test :  0
Running Example  36 

Test example 36 failed

Number of Ran Tests :  7
Number of failed Tests :  6
Number of error Tests :  0
Number of Succeeded T

KeyboardInterrupt: 

## Judge

In [2]:
judgeGenerator = DecisionMaker(judgeChain,globals())
judgeGenerator.generate()

Directory JudgeOutput/ created successfully!
File JudgeOutput/JudgmentLogs.json created successfully!
Running Test Case  0
Example 0  has already passed
Running Test Case  1
generatedJudgement:  <s> [INST] Given the Python code below, its description, and an error-producing test case with the associated error message, determine the cause of the error. Use the information provided to decide whether the error is due to a bug in the Python code itself or in the test case. Provide a reasoned conclusion.

- Code Under Test:



def is_prime(n):
    if n < 2:
        return False
    for k in range(2, n - 1):
        if n % k == 0:
            return False
    return True

- Description of Functionality:
    Return true if a given number is prime, and false otherwise.
    >>> is_prime(6)
    False
    >>> is_prime(101)
    True
    >>> is_prime(11)
    True
    >>> is_prime(13441)
    True
    >>> is_prime(61)
    True
    >>> is_prime(4)
    False
    >>> is_prime(1)
    False

- Error-Induc

TypeError: argument of type 'NoneType' is not iterable

## BugFix

In [ ]:
bugFixGenerator.generate()

True
File BugFixOutput/Cases.json created successfully!
File BugFixOutput/RunningLogs.json created successfully!
Running Example  0 

Test Case 0 Didn't Run Due to Incomplete Response

Test example 0 failed

Number of Ran Tests :  3
Number of failed Tests :  0
Number of error Tests :  0
Number of Succeeded Test :  3
Running Example  1 

Test Case 1 Didn't Run Due to Incomplete Response

Test example 1 failed

Number of Ran Tests :  4
Number of failed Tests :  0
Number of error Tests :  0
Number of Succeeded Test :  4
Running Example  2 

Example 2  has already passed
Running Example  3 

Example 3  has already passed
Running Example  4 

Test Case 4 Didn't Run Due to Incomplete Response

Test example 4 failed

Number of Ran Tests :  7
Number of failed Tests :  0
Number of error Tests :  0
Number of Succeeded Test :  7
Running Example  5 

Test Case 5 Didn't Run Due to Incomplete Response

Test example 5 failed

Number of Ran Tests :  7
Number of failed Tests :  0
Number of error Tests 

## BugFixBen

In [3]:
bugFixGeneratorben.generate()

Running Example  0 

Test example 0 failed due to syntax or indentation or timeout

Number of Ran Tests :  0
Number of failed Tests :  0
Number of error Tests :  1
Number of Succeeded Test :  0
Running Example  1 

Test example 1 succeeded

Number of Ran Tests :  7
Number of Succeeded Test :  7
Number of Succeeded Test :  0
Running Example  2 

Test example 2 succeeded

Number of Ran Tests :  7
Number of Succeeded Test :  7
Number of Succeeded Test :  0
Running Example  3 

Test example 3 succeeded

Number of Ran Tests :  7
Number of Succeeded Test :  7
Number of Succeeded Test :  0
Running Example  4 

Test example 4 failed

Number of Ran Tests :  7
Number of failed Tests :  0
Number of error Tests :  6
Number of Succeeded Test :  1
Running Example  5 

Test example 5 failed

Number of Ran Tests :  6
Number of failed Tests :  0
Number of error Tests :  4
Number of Succeeded Test :  2
Running Example  6 

Test example 6 succeeded

Number of Ran Tests :  11
Number of Succeeded Test :  1

In [ ]:
import openai
# from langchain_openai import OpenAI
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt = PromptTemplate(
            template="How to say {input} in {output_language}:\n",
            input_variables=["input", "output_language"],
            verbose=False)

# llm = ChatPromptTemplate(
llm = ChatOpenAI(temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

chain = prompt | llm

chain.invoke(
    {
        "output_language": "German",
        "input": "I love programming.",
    }
)